## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore', FutureWarning)

We will be using the required dataset for this assignment.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. <br/> 
Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's load the CSV data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
predictors_normalized = (predictors - predictors.mean()) / predictors.std()
predictors_normalized.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
number_of_columns = predictors_normalized.shape[1] # number of predictors
number_of_columns

8

## Import Keras


#### Let's go ahead and import the Keras library


In [11]:
import keras

Using TensorFlow backend.


As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [12]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(number_of_columns,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.


Let's import scikit-learn in order to randomly split the data into a training and test sets


<a id='item34'></a>


In [14]:
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


Splitting the data into a training and test sets by holding 30% of the data for testing


In [15]:
x_training, x_testing, y_training, y_testing = train_test_split(predictors, target, test_size=0.3, random_state=50)

## Train and Test the Network


Let's call the function now to create our model.


In [16]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [17]:
# fit the model
epochs = 100
model.fit(x_training, y_training, epochs=epochs, verbose=1)



Epoch 1/100
 32/721 [>.............................] - ETA: 4s - loss: 19573.9844

2024-08-24 23:48:32.080342: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-08-24 23:48:32.085472: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2494050000 Hz
2024-08-24 23:48:32.086016: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55fce450e8b0 executing computations on platform Host. Devices:
2024-08-24 23:48:32.086059: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-08-24 23:48:32.157024: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

721/721 [==============================] - 0s 343us/step - loss: 11831.9270
Epoch 2/100
721/721 [==============================] - 0s 85us/step - loss: 8452.1398
Epoch 3/100
721/721 [==============================] - 0s 84us/step - loss: 6948.1255
Epoch 4/100
721/721 [==============================] - 0s 101us/step - loss: 5530.0124
Epoch 5/100
721/721 [==============================] - 0s 84us/step - loss: 4318.5239
Epoch 6/100
721/721 [==============================] - 0s 79us/step - loss: 3350.0766
Epoch 7/100
721/721 [==============================] - 0s 62us/step - loss: 2564.5957
Epoch 8/100
721/721 [==============================] - 0s 60us/step - loss: 2061.9211
Epoch 9/100
721/721 [==============================] - 0s 72us/step - loss: 1707.7043
Epoch 10/100
721/721 [==============================] - 0s 59us/step - loss: 1450.5921
Epoch 11/100
721/721 [==============================] - 0s 54us/step - loss: 1250.1205
Epoch 12/100
721/721 [==============================] - 0s 63

Next we need to evaluate the model on the test data.


In [18]:
loss_val = model.evaluate(x_testing, y_testing)
y_prediction = model.predict(x_testing)
loss_val

309/309 [==============================] - 0s 89us/step


106.97428205175308

In [ ]:
Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.


In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_square_error = mean_squared_error(y_testing, y_prediction)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

106.97430825337297 0.0


In [ ]:
total_mean_squared_errors = 50
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    x_training, x_testing, y_training, y_testing = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(x_training, y_training, epochs=epochs, verbose=0)
    MeanSquaredError = model.evaluate(x_testing, y_testing, verbose=0)
    print("Mean Squared Error "+str(i+1)+": "+str(MeanSquaredError))
    y_prediction = model.predict(x_testing)
    mean_square_error = mean_squared_error(y_testing, y_prediction)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("This is the mean with a standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of training epochs: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Squared Error 1: 92.79364379092713
Mean Squared Error 2: 122.67464824787622
Mean Squared Error 3: 121.6436899919726
Mean Squared Error 4: 121.42572609120587
Mean Squared Error 5: 123.35499873979192
Mean Squared Error 6: 109.29540682301938
Mean Squared Error 7: 132.43124769883633
Mean Squared Error 8: 105.28217688501846
Mean Squared Error 9: 119.49960080242465
Mean Squared Error 10: 110.44392632049264
Mean Squared Error 11: 130.26092383622353
Mean Squared Error 12: 100.47939517814365
Mean Squared Error 13: 116.66688722777135
Mean Squared Error 14: 116.62239948754171
Mean Squared Error 15: 112.26866685688303
Mean Squared Error 16: 108.06025774810692
Mean Squared Error 17: 117.2513910682456
Mean Squared Error 18: 105.86800954874279
Mean Squared Error 19: 100.06933606095299
Mean Squared Error 20: 114.14320837718384
Mean Squared Error 21: 98.9348689946542
Mean Squared Error 22: 103.10797980842467
Mean Squared Error 23: 113.37146731873547
Mean Squared Error 24: 106.52487834448954
Mean S